In [ ]:
import pandas as pd
import numpy as np
import json
import networkx as nx
import plotly.graph_objects as go
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap 
import dash
import dash_core_components as dcc
import dash_html_components as html

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
viridis = cm.get_cmap('viridis', 12)
G = nx.random_geometric_graph(nodes.shape[0], 0.125)

In [ ]:
df =pd.read_csv('../data/data.csv')

In [ ]:
df['Caller_node']=df['Caller'].apply(lambda x:list(nodes).index(x))
df['Receiver_node']=df['Receiver'].apply(lambda x:list(nodes).index(x))
df['Dura_color']=(df['Duration']/df['Duration'].max()).apply(viridis)
df['Date']=df['Date'].apply(pd.to_datetime)
df['Dura_color']=(df['Duration']).apply(viridis)

In [ ]:
selected_date='2020-06-18'
df[df['Date']==pd.to_datetime(selected_date)]

In [ ]:
#https://plotly.com/python/network-graphs/#create-edges
#nodes and edges

edges=np.array(df[['Caller','Receiver']])
nodes=np.union1d(df['Caller'].unique(),df['Receiver'].unique())
#adding lines
edge_trace=[]
def add_coords(x): 
    x0,y0=G.nodes[x['Caller_node']]['pos']
    x1,y1=G.nodes[x['Receiver_node']]['pos']
    edge_trace.append(dict(type='scatter',
    x=[x0,x1], y=[y0,y1],
    line=dict(width=0.5, color='rgba'+str(x['Dura_color'])),
    hoverinfo='none',
    mode='lines'))
df.apply(add_coords,axis=1)
## adding points
node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=10,
        line_width=2))
fig = go.Figure(data=edge_trace+[node_trace],
             layout=go.Layout(
                title='<br>Phone Calls Made',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=True,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )


In [ ]:
app.layout = html.Div(children=[
    html.H1(children='CDR Analyser'),

    html.Div(children='''
        Analyse the phone calls between people.
    '''),
    dcc.Graph(
        id='network-plot',
        figure=fig
    )
])

In [ ]:
app.run_server(mode='jupyterlab',debug=True,port=8000)

In [ ]:
df['Date'].unique()